In [1]:
from libs.Environments.SatelliteEnvironment import *
from libs.DRLAgents.DQNAgent import *
from libs.Constants import *

import numpy as np
import torch 

In [2]:
env = CircularOrbit(
    satellite_num=5
)

env.reset()

state = env.get_state()
print(len(state))
num_states = state[0].shape[0]
orbit_num  = env.orbit_num
sat_num    = env.satellite_num
num_action = orbit_num * sat_num

print("ORBIT NUM : ", orbit_num)
print("SAT NUM : ", sat_num)
print("STATE NUM : ", num_states)
print("ACTION NUM : ", num_action)

agent      = DQNAgent(num_states    = num_states,
                      hidden_size   = 10000,
                      num_action    = num_action,
                      batch_size    = 512,
                      learning_rate = 0.01)

agent.train()

losses = []
rewards = []

episodes = 100000

for episode in range(episodes):
    env.reset()

    ep_reward = 0
    ep_loss = 0
    step = 0

    while True:
        # env.plot()
        step += 1
        state = env.get_state()

        action = agent.get_action(state)

        reward, done, info = env.step(action)

        next_state = env.get_state() # 
            
        agent.append_sample(state=state, 
                            action=action, 
                            reward=reward, 
                            next_state=next_state,
                            done=done) # 
        
        ep_reward += reward

        if agent.memory_counter >= 5000:
            loss = agent.train_model()
            ep_loss += loss
            
            if(step % 10 == 0):
                agent.update_target()
            
        if done:
            print(info)
            agent.write_summary(ep_reward, ep_loss, step, episode)
            print("episode: {} , the episode reward is {}".format(episode, ep_reward))
            break

IndexError: tuple index out of range